In [58]:
############################## START of NEW CODE
import numpy as np
import math
sg= np.ones(2)
a = np.ones(2)
b =  [2,3]
c=0
d = [4,5]
e=6
f= [7,8]
w = np.ones((2,2))
A = np.ones((2,2))
 
# params are sg,a,b,c,d,e,f
def px():
  return sg*0.25  ### some fumction of the first two  

 
def sx(sg,a,b,c,d,e,f,w):  ## S(x) as function of sigma_i,d_i,f_i,c,e where i,j runs from 1 to 4
  term1= c+np.einsum('i,i', d, sg) 
  term2 =(e+np.einsum('i,i', f, sg))*1j
  return math.tanh(term1+term2)
####
def Bxa(sg,a,b,c,d,e,f,w):
  return 0.5*sg
def Bxb (sg,a,b,c,d,e,f,w):
  return 0.5*np.array(list(map(math.tanh, b+np.einsum('ij,i->j', w, sg))))
def Bxc (sg,a,b,c,d,e,f,w):
  return (1/( sx(sg,a,b,c,d,e,f,w))- sx(sg,a,b,c,d,e,f,w))
def Bxd (sg,a,b,c,d,e,f,w):
  return  (1/( sx(sg,a,b,c,d,e,f,w))- sx(sg,a,b,c,d,e,f,w))*sg
def Bxe (sg,a,b,c,d,e,f,w):
  return (1/( sx(sg,a,b,c,d,e,f,w))- sx(sg,a,b,c,d,e,f,w))*1j

def Bxf (sg,a,b,c,d,e,f,w):
  return (1/( sx(sg,a,b,c,d,e,f,w))- sx(sg,a,b,c,d,e,f,w))*1j*sg
def Bxw (sg,a,b,c,d,e,f,w):
  term1 = 0.5*np.array(list(map(math.tanh, b+np.einsum('ij,i->j', w, sg))))
  term2 = sg
  return np.einsum('j,i->ij',term1,term2)
######
def Avx(Bxi,sg,a,b,c,d,e,f,w):
  term1=Bxi(sg,a,b,c,d,e,f,w)
  term2= sx(sg,a,b,c,d,e,f,w)*np.array(list(map(math.sqrt, px())))
  term3 = np.array(list(map(abs, term2)))
  term4 = np.square(term3)
  
  if Bxi == Bxc or Bxi == Bxe :
    return Bxi(sg,a,b,c,d,e,f,w)
  elif Bxi == Bxw:
    return np.einsum('ij,i->ij',term1,term4)/ np.einsum('i->',term4) ##revision
  else:
    return np.einsum('i,i->i',term1,term4)/ np.einsum('i->',term4) ### revision
##########
def Avxx(Bxi,sg,a,b,c,d,e,f,w):
  term1 = Bxi(sg,a,b,c,d,e,f,w)
  term2= sx(sg,a,b,c,d,e,f,w)*np.array(list(map(math.sqrt, px())))
  term3 = np.array(list(map(abs, term2)))
  term4 = np.square(term3)
  term5 = np.array(list(map(np.conj, term2)))
 
  if Bxi == Bxc or Bxi == Bxe :
    return Bxi(sg,a,b,c,d,e,f,w)*(np.einsum('i,ij,j->',term5,A,term2))/np.einsum('i->',term4)
  elif Bxi == Bxw:
    return np.einsum('i,ij,jk,j->jk',term5,A,term1,term2)/np.einsum('i->',term4) ####need to be revised I guess what to sum over
  else:
    return np.einsum('i,ij,j,j->j',term5,A,term1,term2)/np.einsum('i->',term4)  ##same comment
def AvA (sg,a,b,c,d,e,f,w):
  term2= sx(sg,a,b,c,d,e,f,w)*np.array(list(map(math.sqrt, px())))
  term3 = np.array(list(map(abs, term2)))
  term4 = np.square(term3)
  term5 = np.array(list(map(np.conj, term2)))
  return (np.einsum('i,ij,j->',term5,A,term2))/np.einsum('i->',term4)




 

def gd(grad, sg,a,b,c,d,e,f,w, n_epochs=3000, eta=0.01): 
    #This is a simple optimizer
    term1=np.append(np.concatenate((a,b)),c) 
    term2 = np.append(term1,d)
    term3=np.append(term2,e)
    term4 = np.append(term3,f)
    params = np.append(term4,w)

    param_traj=np.zeros([n_epochs+1,2])
    param_traj[0,]=params
    v=0;
    for j in range(n_epochs):
         v=eta*(np.array(grad(params)))
         params=params-v
         param_traj[j+1,]=params
    return params

def partialAx(Bxi,sg,a,b,c,d,e,f,w): 
  term1 = Avxx(Bxi,sg,a,b,c,d,e,f,w) 
  term2 = term1 + np.array(list(map(np.conj, term1)))
  term3 = Avx(Bxi,sg,a,b,c,d,e,f,w) 
  term4 = AvA(sg,a,b,c,d,e,f,w)*(term3+ np.array(list(map(np.conj, term3))))
  return term2+term4

 
print(Avxx(Bxa,sg,a,b,c,d,e,f,w))
print(Avxx(Bxb,sg,a,b,c,d,e,f,w))
print(Avxx(Bxc,sg,a,b,c,d,e,f,w))
print(Avxx(Bxd,sg,a,b,c,d,e,f,w))
print(Avxx(Bxe,sg,a,b,c,d,e,f,w))
print(Avxx(Bxf,sg,a,b,c,d,e,f,w))
print(Avxx(Bxw,sg,a,b,c,d,e,f,w))
print("F")
print(grad(Bxw,sg,a,b,c,d,e,f,w))

 
############################## END of NEW CODE

[0.5 0.5]
[0.49966465 0.4999546 ]
1.2183983777447338e-07
[6.09199189e-08 6.09199189e-08]
1.2183983777447338e-07j
[0.+6.09199189e-08j 0.+6.09199189e-08j]
[[0.49966465 0.4999546 ]
 [0.49966465 0.4999546 ]]
F
[[1.9986586  1.99981841]
 [1.9986586  1.99981841]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: ComplexWarning: Casting complex values to real discards the imaginary part
